In [1]:
import pandas as pd
import numpy as np
import pymssql
mssql_conn = pymssql.connect(server='epdm', user='sa', password='Pdmepdme1!',database='ePDM_Elcam')

In [ ]:
query='select * from dbo.UserRevs'
UserRevs=pd.read_sql(query,mssql_conn)

In [ ]:
query2='select * from dbo.Documents'
Documents=pd.read_sql(query2,mssql_conn)
Documents

In [ ]:
q='select * from dbo.Status'
status=pd.read_sql(q,mssql_conn)
doc_status=pd.merge(Documents,status,left_on='CurrentStatusID',right_on='StatusID')
doc_status
# doc+status

In [ ]:
q="""select Filename  ,s.Name ,ur.Comment 
from Documents d 
full join Status s on s.StatusID =d.CurrentStatusID 
full join UserRevs ur on d.DocumentID =ur.DocumentID 
where Filename like '%.sldprt' or Filename like '%.slddrw%'"""
df=pd.read_sql(q,mssql_conn)
#drop all rows where the comment is None

df

In [ ]:
query3="""SELECT DISTINCT ValueCache
FROM VariableValue VV

INNER JOIN

(SELECT DocumentID, ConfigurationID, VariableID, MAX(RevisionNo) AS RevMax

FROM VariableValue
WHERE VariableID=(SELECT VariableID FROM Variable WHERE VariableName= 'PrcdrOrderNum&Subject')
GROUP BY DocumentID,ConfigurationID,VariableID) T

ON VV.DocumentID=T.DocumentID
AND VV.ConfigurationID=T.ConfigurationID
AND VV.VariableID=T.VariableID
AND VV.RevisionNo=T.RevMax
And VV.ValueCache != ''

INNER JOIN

(SELECT D.DocumentID
FROM Projects P,
          DocumentsInProjects P1,
          Documents D,
          Revisions R,
          Status S,
          FileExtension E
WHERE P1.ProjectID=P.ProjectID AND
	  P1.DocumentID=D.DocumentID AND 
	  P.Path LIKE '%' AND
	  D.DocumentID=R.DocumentID AND
D.DocumentID!=55910 AND
	  D.Filename LIKE '%'AND
	  E.ExtensionID=D.ExtensionID AND
	  R.RevNr=D.LatestRevisionNo AND
	  D.CurrentStatusID=S.StatusID AND
	  S.Name NOT IN ('Obsolete') AND
	  D.Deleted='FALSE') Z

ON VV.DocumentID=Z.DocumentID

ORDER BY ValueCache
"""



In [ ]:
res=pd.read_csv('test1.csv')
for i in range(len(res)):
    row=res.iloc[i]
    print(row['ValueCache'])
# res.to_csv('test1.csv',index=False)

In [ ]:
# q1="select * from VariableValue"
# q2="select * from Variable"
# VV=pd.read_sql(q1,mssql_conn)
# V=pd.read_sql(q2,mssql_conn)


In [ ]:
# tempV=V.to_csv('tempV.csv',index=False)
# tempVV=VV.to_csv('tempVV.csv',index=False)

In [ ]:
v=pd.read_csv('tempV.csv')
vv=pd.read_csv('tempVV.csv')


In [ ]:
all_vars_ids=set(v['VariableID'])
all_vars_values_ids=set(vv['VariableID'])


In [ ]:
diff=all_vars_ids-all_vars_values_ids
res={}
for i in diff:
    if i>44:
        res[i]=v[v['VariableID']==i]['VariableName'].iloc[0]
res=pd.DataFrame({'VariableID':list(res.keys()),'VariableName':list(res.values())})
res.to_csv('res.csv',index=False)
res

In [ ]:
q="select * from CardListValues"
card_list_values=pd.read_sql(q,mssql_conn)


In [ ]:
all_ids_values_in_card=set(card_list_values['CardListId'])
print(card_list_values[card_list_values['CardListId']==1275]['Value'].iloc[0])